In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


def plot_description(text):
    print(f"\nDescription:\n{text}\n")


iris = load_iris(as_frame=True)
X = iris.data[["sepal length (cm)", "sepal width (cm)"]][50:]
y = iris.target[50:]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

def plot_boundary_and_confusion(title, clf_model):
    # build and fit pipeline
    clf = Pipeline(steps=[("scaler", StandardScaler()), ("clf", clf_model)])
    clf.fit(X_train, y_train)

    # prepare 2x2 figure: row0 = train, row1 = test
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axs = axes.reshape(2, 2)

    # scores for informative titles
    score_train = clf.score(X_train, y_train)
    score_test = clf.score(X_test, y_test)

    for row, (X_eval, y_eval, set_name, score) in enumerate(
        [(X_train, y_train, "Train Set", score_train), (X_test, y_test, "Test Set", score_test)]
    ):
        ax_bound = axs[row, 0]
        ax_cm = axs[row, 1]

        # Decision boundary (background) using full feature grid X
        DecisionBoundaryDisplay.from_estimator(
            clf,
            X,
            response_method="predict",
            xlabel=iris.feature_names[0],
            ylabel=iris.feature_names[1],
            alpha=0.4,
            cmap=plt.cm.RdBu,
            ax=ax_bound,
        )

        # scatter the evaluation points on top of the boundary
        for label, color, marker in zip([1, 2], ["C1", "C0"], ["o", "s"]):
            sel = (y_eval == label)
            ax_bound.scatter(
                X_eval.loc[sel, X_eval.columns[0]],
                X_eval.loc[sel, X_eval.columns[1]],
                c=color,
                marker=marker,
                edgecolor="k",
                s=80 if label == 2 else 50,
                zorder=2 if label == 2 else 3,
                label=f"target {label-1} ({iris.target_names[label]})",
                alpha=0.9,
            )

        ax_bound.set_xlim([4.5, 8])
        ax_bound.set_ylim([1.8, 4])
        ax_bound.legend(loc="upper left")
        ax_bound.set_title(f"{title}\n" + r"$\mathbf{" + set_name + "}$")

        # Confusion matrix on the right for the same eval set
        y_pred = clf.predict(X_eval)
        acc = accuracy_score(y_eval, y_pred)
        disp_cm = ConfusionMatrixDisplay.from_predictions(
            y_eval,
            y_pred,
            display_labels=[iris.target_names[1], iris.target_names[2]],
            cmap=plt.cm.Blues,
            ax=ax_cm,
        )

        # increase font size of the matrix numbers (compat across sklearn versions)
        tick_fontsize = 16
        try:
            for txt in disp_cm.text_.ravel():
                txt.set_fontsize(tick_fontsize)
        except Exception:
            for txt in disp_cm.ax_.texts:
                txt.set_fontsize(tick_fontsize)

        # bold accuracy in the confusion matrix title
        acc_text = f"Accuracy: {acc:.2f}"
        ax_cm.set_title(
            f"{title}\n" + r"$\mathbf{" + set_name + " - " + acc_text + "}$"
        )

    plt.tight_layout()
    plt.show()
